In [147]:
import pandas as pd
import numpy as np
import wikipedia
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from sentence_transformers import CrossEncoder
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch

In [148]:
# Wikipedia Data Scraping (Tagalog)
wikipedia.set_lang("tl")
try:
    pages = ["Kultura ng Pilipinas", "Lutuing Pilipino", "Kasaysayan ng Pilipinas", "Talaan ng mga lungsod at bayan sa Pilipinas"]
    texts = [wikipedia.page(page).content for page in pages]
except wikipedia.exceptions.DisambiguationError as e:
    print(f"Disambiguation error. Options: {e.options}")
except Exception as e:
    print(f"Error: {e}")

In [149]:
# Dense Embeddings (BGE-M3)
embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    encode_kwargs={"normalize_embeddings": True}
)

In [150]:
# Chunking (Semantic)

semantic_splitter = SemanticChunker(
    embeddings=embedding_model,
    breakpoint_threshold_type="standard_deviation",
    breakpoint_threshold_amount=0.75,
    buffer_size=1
)

In [151]:
print("[INFO] Performing semantic chunking...")

# Apply chunking to each page and concatenate results
chunks = []
for page in texts:
    chunks.extend(semantic_splitter.split_text(page))

print(f"[INFO] Total chunks generated: {len(chunks)}")

# Print
for i, chunk in enumerate(chunks[:2], start=1):
    print(f"\nChunk {i}:\n{chunk[:300]}...\n{'-'*60}")

[INFO] Performing semantic chunking...
[INFO] Total chunks generated: 174

Chunk 1:
Ang kultura ng Pilipinas o kalinangan ng Pilipinas ay pinaghalong impluwensiya ng mga katutubong tradisyon at mga kultura ng mga unang mangangalakal at mananakop nito noon. Ang pananakop ng mga Kastila sa Pilipinas, sa pamamahala ng España, na tumagal ng mahigit 333 taon, ay may malaking kontribusyo...
------------------------------------------------------------

Chunk 2:
Bilang halimbawa, bawat taon, ang mga bayan sa buong bansa, ay nagsasagawa ng malalaking Pista, nagpapaalala sa mga Santong Patron ng mga bayan, barangay, o ng mga distrito. Ang mga Pista ay kadalasang may patimpalak sa katutubong pagsayaw, at sa ibang lugar ay mayroon pang sabungan. Ang mga ganiton...
------------------------------------------------------------


In [152]:
# Chroma Vector Store (Dense Retrieval)
vectorstore = Chroma.from_texts(
    texts=chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db",
    collection_name="filipino_culture"
)

In [153]:
# BM25 Retriever
bm25_retriever = BM25Retriever.from_texts(chunks)
bm25_retriever.k = 3

In [154]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[
        vectorstore.as_retriever(search_kwargs={"k": 3}),
        bm25_retriever
    ],
    weights=[0.7, 0.3]
)

In [158]:
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")

def rerank_with_cross_encoder(query, docs, top_n=2):
    pairs = [[query, doc.page_content] for doc in docs]
    scores = cross_encoder.predict(pairs)
    scored_docs = list(zip(scores, docs))
    scored_docs.sort(key=lambda x: x[0], reverse=True)
    return scored_docs[:top_n] 

d:\Thesis\DatasetDump\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\herna\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regula

In [159]:
# Test Output
query = "Ano-ano ang mga ulam na dinala ng mga Kastila sa Pilipinas?"

# Retrieve documents using hybrid retriever
initial_results = ensemble_retriever.get_relevant_documents(query)

# Rerank using Cross-Encoder
reranked = rerank_with_cross_encoder(query, initial_results, top_n=2)

for i, (score, doc) in enumerate(reranked, start=1):
    print(f"\nRank {i} | Score: {score:.4f}")
    print("-" * 60)
    print(doc.page_content.strip()[:300])
    print("-" * 60)


Rank 1 | Score: 6.7966
------------------------------------------------------------
Kabilang sa mga ibang sikat na ulam na may impluwensyang Timog-silangang Asyano at Kastila ang apritada, asado, chorizo, empanada, mani, paksiw, pandesal, pescado frito (pinritong isda), sisig, torta, kare-kare, kilawen, pinakbet, pinapaitan, at sinigang. Waring di-nakagaganang kainin sa paletang Ka
------------------------------------------------------------

Rank 2 | Score: 5.2829
------------------------------------------------------------
Ang kakulangan ng mga sandata ang naging sanhi ng pagkatalo ng mga Pilipinong sundalo laban sa mga Amerikano sa mga pangunahing labanan ngunit ang mga Pilipino ay nagwagi sa mga labanang gerilya. Ang Malolos, na kabisera ng pamahalaang rebolusyonaryo, ay nakuha ng mga Amerikano noong ika-31 ng Marso
------------------------------------------------------------
